## Prerequisites

Before running this notebook, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 🏦 Azure AI Agent with Explicit Configuration - Investment Portfolio Management

This notebook demonstrates creating Azure AI Agents with **explicit configuration settings** rather than relying on environment variable defaults. This is particularly useful in scenarios where you need precise control over agent configuration for compliance and governance.

## Features Covered
- ✅ Explicit configuration of Azure AI client settings
- ✅ Direct specification of project endpoint and model deployment  
- ✅ Custom agent naming and settings
- ✅ Use case: Investment portfolio analysis with configurable parameters
- ✅ Production-ready configuration patterns

## Scenario: Investment Advisory Service
A configurable investment advisor that can be deployed with different model settings based on client tier (standard vs. premium) or compliance requirements.

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Microsoft Foundry project with deployed models
- A `.env` file with the required configuration (see below)

### Required Environment Variables
```
AI_FOUNDRY_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o-mini
```

## 📚 Import Required Libraries

In [ ]:
import os
from typing import Annotated

from agent_framework.azure import AzureAIProjectAgentProvider
from azure.identity.aio import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify required environment variables
required_vars = ["AI_FOUNDRY_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
for var in required_vars:
    if var not in os.environ:
        raise ValueError(f"Missing required environment variable: {var}")
    print(f"✅ {var} = {os.environ[var][:50]}...")

## 💼 Define Investment Tools

In [ ]:
# Investment Advisory Functions

def get_portfolio_allocation(
    client_id: Annotated[str, Field(description="The client identifier")],
    risk_profile: Annotated[str, Field(description="Risk profile: conservative, moderate, aggressive")],
) -> str:
    """Get recommended portfolio allocation based on client risk profile."""
    allocations = {
        "conservative": {
            "bonds": 60,
            "stocks": 25,
            "cash": 10,
            "alternatives": 5
        },
        "moderate": {
            "bonds": 40,
            "stocks": 45,
            "cash": 10,
            "alternatives": 5
        },
        "aggressive": {
            "bonds": 20,
            "stocks": 60,
            "cash": 5,
            "alternatives": 15
        }
    }
    
    profile = risk_profile.lower()
    if profile not in allocations:
        return f"Unknown risk profile: {risk_profile}. Please specify conservative, moderate, or aggressive."
    
    allocation = allocations[profile]
    return f"""Portfolio Allocation for Client {client_id} ({profile.capitalize()} profile):
    - Bonds: {allocation['bonds']}%
    - Stocks: {allocation['stocks']}%
    - Cash: {allocation['cash']}%
    - Alternatives: {allocation['alternatives']}%"""


def get_investment_performance(
    portfolio_type: Annotated[str, Field(description="Portfolio type: growth, income, balanced")],
    time_period: Annotated[str, Field(description="Time period: ytd, 1y, 3y, 5y")],
) -> str:
    """Get historical performance metrics for investment portfolios."""
    performance_data = {
        "growth": {"ytd": 12.5, "1y": 18.3, "3y": 45.2, "5y": 82.1},
        "income": {"ytd": 4.2, "1y": 6.8, "3y": 15.4, "5y": 28.7},
        "balanced": {"ytd": 8.1, "1y": 11.9, "3y": 28.5, "5y": 52.3}
    }
    
    ptype = portfolio_type.lower()
    period = time_period.lower()
    
    if ptype not in performance_data:
        return f"Unknown portfolio type: {portfolio_type}"
    if period not in performance_data[ptype]:
        return f"Unknown time period: {time_period}"
    
    return f"""Performance for {portfolio_type.capitalize()} Portfolio ({period.upper()}):
    - Return: {performance_data[ptype][period]}%
    - Benchmark S&P 500 comparison: {'+' if performance_data[ptype][period] > 10 else ''}{performance_data[ptype][period] - 10:.1f}% vs benchmark"""


def calculate_retirement_projection(
    current_age: Annotated[int, Field(description="Client's current age")],
    retirement_age: Annotated[int, Field(description="Target retirement age")],
    current_savings: Annotated[float, Field(description="Current retirement savings")],
    monthly_contribution: Annotated[float, Field(description="Monthly contribution amount")],
) -> str:
    """Project retirement savings based on current trajectory."""
    years_to_retirement = retirement_age - current_age
    annual_return = 0.07  # Assumed 7% average annual return
    
    # Future value calculation with compound interest
    future_value = current_savings * ((1 + annual_return) ** years_to_retirement)
    contribution_value = monthly_contribution * 12 * (((1 + annual_return) ** years_to_retirement - 1) / annual_return)
    total_projected = future_value + contribution_value
    
    return f"""Retirement Projection:
    - Current Age: {current_age}
    - Target Retirement Age: {retirement_age}
    - Years to Retirement: {years_to_retirement}
    - Current Savings: ${current_savings:,.2f}
    - Monthly Contribution: ${monthly_contribution:,.2f}
    - Projected Value at Retirement: ${total_projected:,.2f}
    - Assumed Annual Return: 7%"""

## 🔧 Create Agent with Explicit Configuration

This demonstrates creating an agent with **explicit settings** passed directly to the provider, rather than relying solely on environment variables. This gives you fine-grained control over agent configuration.

In [ ]:
async def investment_advisor_explicit_config() -> None:
    """
    Create an Investment Advisor agent with explicit configuration settings.
    This pattern is useful when you need precise control over deployment parameters.
    """
    print("=== 🏦 Investment Advisor with Explicit Configuration ===")
    
    # Explicit configuration values (could come from config files, secrets manager, etc.)
    explicit_endpoint = os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"]
    explicit_model = os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"]
    
    print(f"📍 Endpoint: {explicit_endpoint}")
    print(f"🤖 Model: {explicit_model}")
    
    # Create provider with EXPLICIT settings using the new API
    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(
            project_endpoint=explicit_endpoint,  # Explicit endpoint
            model=explicit_model,  # Explicit model
            credential=credential,
        ) as provider,
    ):
        # Create the investment advisor agent
        # Note: Agent names must use hyphens (not underscores), start/end with alphanumeric
        agent = await provider.create_agent(
            name="investment-advisor-explicit-config",
            instructions="""You are a professional Investment Advisor. You help clients with:
            - Portfolio allocation recommendations based on risk profile
            - Investment performance analysis
            - Retirement planning projections
            
            Always provide clear explanations and remind clients that past performance 
            does not guarantee future results. Be professional and thorough.""",
            tools=[
                get_portfolio_allocation,
                get_investment_performance,
                calculate_retirement_projection,
            ],
        )
        
        print(f"\n✅ Agent created: {agent.name}")
        
        # Test with portfolio allocation query
        query = "I'm client C12345 with a moderate risk profile. What portfolio allocation do you recommend?"
        print(f"\n💬 Client: {query}")
        
        result = await agent.run(query)
        print(f"\n📊 Advisor: {result}")

## 🚀 Execute the Investment Advisor

In [ ]:
await investment_advisor_explicit_config()

## 🔄 Comprehensive Investment Consultation

In [ ]:
async def comprehensive_consultation() -> None:
    """
    Demonstrates a full investment consultation using multiple tools.
    """
    print("=== 📊 Comprehensive Investment Consultation ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(
            project_endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            credential=credential,
        ) as provider,
    ):
        agent = await provider.create_agent(
            name="comprehensive-investment-advisor",
            instructions="""You are an expert Investment Advisor providing comprehensive financial guidance.
            Use all available tools to give thorough, personalized advice.
            Always include appropriate disclaimers about investment risks.""",
            tools=[
                get_portfolio_allocation,
                get_investment_performance,
                calculate_retirement_projection,
            ],
        )
        
        # Multi-turn consultation
        queries = [
            "I'm 35 years old, looking to retire at 65. I have $150,000 saved and can contribute $1,500/month. What's my projection?",
            "Given that projection, should I go with an aggressive or moderate portfolio? Show me the performance comparison for both.",
            "Based on all this, give me your recommended portfolio allocation for client ID INV-2024-35."
        ]
        
        for i, query in enumerate(queries, 1):
            print(f"\n{'='*60}")
            print(f"📩 Question {i}: {query}")
            result = await agent.run(query)
            print(f"\n💼 Advisor: {result}")

## 🚀 Run Comprehensive Consultation

In [ ]:
await comprehensive_consultation()

## 📝 Key Takeaways

### Benefits of Explicit Configuration

1. **Fine-Grained Control**: Pass specific endpoints and model names directly instead of relying on environment variables

2. **Multi-Environment Support**: Easily switch between dev/staging/production configurations:
   ```python
   async with (
       AzureCliCredential() as credential,
       AzureAIProjectAgentProvider(
           project_endpoint=config.get_endpoint(environment),
           model=config.get_model(tier),
           credential=credential,
       ) as provider,
   ):
   ```

3. **Dynamic Configuration**: Change settings at runtime based on client requirements or compliance needs

4. **Testing Flexibility**: Mock or override settings during testing without changing environment variables

### Use Cases for Explicit Configuration

- **Client Tier Routing**: Route premium clients to more capable models
- **Compliance Requirements**: Use specific model deployments approved for regulated data
- **Regional Deployment**: Connect to region-specific endpoints for data residency
- **A/B Testing**: Test different model versions with different client segments

### ⚠️ Disclaimer
This example uses simulated data for demonstration purposes. In production:
- Connect to actual data sources
- Implement proper security and authentication
- Follow all regulatory requirements
- Include appropriate risk disclosures